In [ ]:
import pandas as pd

df = pd.read_csv("week9-sample.csv", sep='\t')

# x = df.groupby('Kisi')['Dakika'].apply(pd.rolling_mean, 2, min_periods=1)
x = df.groupby('Kisi')['Dakika'].rolling(2).sum().reset_index(0,drop=True)
y = df.groupby('Kisi')['Dakika'].rolling(3).sum().reset_index(0,drop=True)
print(y)

In [ ]:
import pandas as pd
import numpy as np
telecom = pd.read_csv("telecom_churn_data.csv")

# Where summation of columns = 0 then churn =1  else 0
telecom['churn']= np.where(telecom[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9',\
									'vol_3g_mb_9']].sum(axis=1)== 0,1,0)

# Removing all features having ‘ _9’, etc. in their names
telecom.drop(telecom.filter(regex='_9|sep', axis = 1).columns, axis=1,inplace=True)

In [ ]:
del telecom['mobile_number']

telecom = telecom.fillna(0)

del telecom['sachet_2g_6']
del telecom['sachet_2g_7']
del telecom['sachet_2g_8']
del telecom['total_rech_amt_6']
del telecom['total_rech_amt_7']
del telecom['total_rech_amt_8']

columns = list(telecom.columns)

In [ ]:
#! ELIMINATING INTERCORRELATED
for x1 in columns:
	for x2 in columns:
		if str(telecom.dtypes[x1]) in ['int64', 'float64']:
			if str(telecom.dtypes[x2]) in ['int64', 'float64']:
				if x1 > x2:
					if x1 != 'churn' and x2 != 'churn':
						c = telecom[x1].corr(telecom[x2])
						if abs(c) > 0.95:
							print(x1, x2, c)

In [ ]:
#! ELIMINATING COLUMNS WITH VERY LOW CORRELATION
for x in columns:
	if str(telecom.dtypes[x]) in ['int64', 'float64']:
		c = telecom[x].corr(telecom['churn'])
		if abs(c) < 0.05:
			del telecom[x]
			print(c, x)

In [ ]:

degisken_index = 0
for x1 in columns:
	for x2 in columns:
		if str(telecom.dtypes[x1]) in ['int64', 'float64']:
			if str(telecom.dtypes[x2]) in ['int64', 'float64']:
				if x1 > x2:
					if x1 != 'churn' and x2 != 'churn':
						#for t in [1,2,5,10,50,100,1000]:
							degisken_index += 1
							column = 'compare' + str(degisken_index)

							#telecom[column] = telecom[ [x1, x2] ].apply(lambda v: safeDivide(v), axis = 1 ) 

							telecom[column] = telecom[x1] > telecom[x2]
							telecom[column] = telecom[column].astype(int)

							c1 = abs(telecom[x1].corr(telecom['churn']))
							c2 = abs(telecom[x2].corr(telecom['churn']))

							correlation = telecom[column].corr(telecom['churn'])
							if abs(correlation) > 0.50 and abs(correlation) > 0.10 + c1 and abs(correlation) > 0.10 + c2:
								print(correlation, x1, '>', x2)
							else:
								del telecom[column]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

for c in telecom:
	if str(telecom.dtypes[c]) == 'object':
		del telecom[c]

In [ ]:

data = telecom.copy()
y = data['churn']
x = data
del x['churn']

clf = RandomForestClassifier(max_depth=6, random_state=0)
clf.fit(x, y)
print(clf.score(x,y))

importances = list(clf.feature_importances_)
for i in range(len(list(x.columns))):
    #if importances[i] > 0.01:
    print(i, x.columns[i], importances[i])


In [1]:
import numpy as np 
df['churn'] = np.where(df[['A','B','C']].sum(axis=1)== 0, 1, 0)
